In [24]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from signSGD import SignSGD

In [25]:
# Define the neural network architecture
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # 2 output classes for binary classification

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [26]:
# Load the dataset
data = pd.read_csv("./adult.csv")

print(data.head(3))

   age  workclass  fnlwgt   education  educational-num      marital-status  \
0   25    Private  226802        11th                7       Never-married   
1   38    Private   89814     HS-grad                9  Married-civ-spouse   
2   28  Local-gov  336951  Assoc-acdm               12  Married-civ-spouse   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   
1    Farming-fishing      Husband  White   Male             0             0   
2    Protective-serv      Husband  White   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
1              50  United-States  <=50K  
2              40  United-States   >50K  


In [27]:
# Preprocessing the data
# Encode categorical variables
label_encoders = {}
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col])

# Map income column to binary values
data['income'] = data['income'].map({'<=50K': 0, '>50K': 1})

In [28]:
# Splitting the data into features and target variable
X = data.drop('income', axis=1)
y = data['income']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [29]:
# Define model, loss function, and optimizer
input_dim = X_train.shape[1]
model = SimpleNN(input_dim)
criterion = nn.CrossEntropyLoss()

# Training loop
def train(model, optimizer, criterion, X_train, y_train):
    model.train()
    if optimizer:
        optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    if optimizer:
        loss.backward()
        optimizer.step()
    return loss.item()

# Evaluation function
def evaluate(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test, predicted)
    return accuracy

In [31]:
# MACROS
NUM_EPOCH = 100
LEARNING_RATE = 0.001

# Select optimizer
ADAM = False
ADAGRAD = False
SIGN_SGD = True

if ADAM:
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    print(f"\nTraining with Adam optimizer:")
elif ADAGRAD:
    optimizer = optim.Adagrad(model.parameters(), lr=LEARNING_RATE)
    print(f"\nTraining with Adagrad optimizer:")
elif SIGN_SGD:
    optimizer = SignSGD(model.parameters(), lr=LEARNING_RATE)
    print(f"\nTraining with SignSGD optimizer:")
else:
    optimizer = None
    print(f"\nTraining with no optimizer:")



# Train and evaluate model
for epoch in range(NUM_EPOCH):
    loss = train(model, optimizer, criterion, X_train_tensor, y_train_tensor)
    accuracy = evaluate(model, X_test_tensor, y_test_tensor)
    print(f"Epoch {epoch+1}: Loss={loss:.4f}, Accuracy={accuracy:.4f}")


Training with SignSGD optimizer:
Epoch 1: Loss=0.6229, Accuracy=0.7705
Epoch 2: Loss=0.6229, Accuracy=0.7705
Epoch 3: Loss=0.6229, Accuracy=0.7705
Epoch 4: Loss=0.6229, Accuracy=0.7705
Epoch 5: Loss=0.6229, Accuracy=0.7705
Epoch 6: Loss=0.6229, Accuracy=0.7705
Epoch 7: Loss=0.6229, Accuracy=0.7705
Epoch 8: Loss=0.6229, Accuracy=0.7705
Epoch 9: Loss=0.6229, Accuracy=0.7705
Epoch 10: Loss=0.6229, Accuracy=0.7705
Epoch 11: Loss=0.6229, Accuracy=0.7705
Epoch 12: Loss=0.6229, Accuracy=0.7705
Epoch 13: Loss=0.6229, Accuracy=0.7705
Epoch 14: Loss=0.6229, Accuracy=0.7705
Epoch 15: Loss=0.6229, Accuracy=0.7705
Epoch 16: Loss=0.6229, Accuracy=0.7705
Epoch 17: Loss=0.6229, Accuracy=0.7705
Epoch 18: Loss=0.6229, Accuracy=0.7705
Epoch 19: Loss=0.6229, Accuracy=0.7705
Epoch 20: Loss=0.6229, Accuracy=0.7705
Epoch 21: Loss=0.6229, Accuracy=0.7705
Epoch 22: Loss=0.6229, Accuracy=0.7705
Epoch 23: Loss=0.6229, Accuracy=0.7705
Epoch 24: Loss=0.6229, Accuracy=0.7705
Epoch 25: Loss=0.6229, Accuracy=0.7705
